### Стенд

Рабочие станции развернуты на Virtual Box, подключены к сети ``192.168.56.0/24``. Работа была проделана в рамках исследования, по-этому содержимое ключевых файлов могут быть опубликованы в данной статье. 

1. Центр сертификации (Certificate authority - CA), развернута на VM с ОС ubuntu 18.04.
    - название виртуальной машины (ВМ): ``CA``
    - ip: ``192.168.56.110``
    - доменное имя: ``ca.local``
    - пользователь: ``ca``
    - пара ключей для продписи сертификатов хостов: ``ca_host_key``, ``ca_host_key.pub``
    - пара ключей для продписи сертификатов пользователей: ``ca_user_key``, ``ca_user_key.pub``

2. SSH cервер/клиент №1, развернут на VM с ОС ubuntu 18.04. 
    - название ВМ: ``server1``
    - ip: ``192.168.56.111``
    - доменное имя: ``u1.local``
    - пользователь: ``u1``
    
3. SSH cервер/клиент №2, развернут на VM с ОС ubuntu 18.04. 
    - название ВМ: ``server2``
    - ip: ``192.168.56.112``
    - доменное имя: ``u2.local``
    - пользователь: ``u2``
    - пара ключей и сетрификат пользователя для для авторизации на сервере: ``id_rsa``, ``id_rsa.pub``, ``id_rsa-cert.pub``  
 


### Подготовка стенда

1. Для того чтобы указанные выше ip адреса были на всегда закреплены за соотвествующими машинами, необходимо в настройках систем выставить ip адреса вручную.

2. Прописать доменные имена для каждого ip адреса в настройках системы каждой виртуальной машины (на всех трех хостах). В *nix-машинах, обычно, это пропысывается в файле ``/etc/hosts``. Содержимое файла должно выглядеть так:

        #
        127.0.0.1	localhost
        ...
        192.168.56.110	ca.local	ca
        192.168.56.111	u1.local	u1
        192.168.56.112	u2.local	u2
        ...
        #

### Генерация ключей Центра Сертификации (CA).

На виртуальной машине ``CA``, назначенном центром сертификации, необходимо создать два ключа для подписании сертификатов хостов и пользователей. Эти ключи, которым должны доверять все остальные хосты. Например, назовем их: ``ca_host_key`` и ``ca_user_key``.

Предварительно создадим директорию, где будут хранится ключи Центра Сертификации:

        sudo -i
        mkdir /etc/ssh_ca
        cd /etc/ssh_ca/


Для того чтобы создать ключ подписи сертификатов хостов, необходимо выполнить следующую команду от имени пользователя ``root``:


        ssh-keygen -q -b 4096 -f ca_host_key


Для того чтобы создать ключ подписи сертификатов пользователей, необходимо выполнить следующую команду от имени пользователя ``root``:


        ssh-keygen -q -b 4096 -f ca_user_key


На виртуальной машине ``CA`` необходимо создать сертификат открытого ключа хоста Центра Сертификации.

Предварительно надем открытый ключ хоста:

        root@ca:/etc/ssh_ca# ls -l /etc/ssh/ssh_host_rsa_key*
        -rw------- 1 root root 1675 сен  2 10:12 /etc/ssh/ssh_host_rsa_key
        -rw-r--r-- 1 root root  389 сен  2 10:12 /etc/ssh/ssh_host_rsa_key.pub

Для того стобы создать сетрификат открытого ключа хоста ВМ ``CA``, необходимо выполнить следующую команду от имени пользователя ``root``:

        ssh-keygen -s ca_host_key -I ca_host_key.local -h -n ca,ca.local -V -1w:+54W /etc/ssh/ssh_host_rsa_key.pub


Получили сертификат ``ssh_host_rsa_key-cert.pub``, можно посмотреть в директории: 

        root@ca:/etc/ssh_ca# ls -l /etc/ssh/ssh_host_rsa_key*
        -rw------- 1 root root 1675 сен  2 10:12 /etc/ssh/ssh_host_rsa_key
        -rw-r--r-- 1 root root 2026 сен  2 13:36 /etc/ssh/ssh_host_rsa_key-cert.pub
        -rw-r--r-- 1 root root  389 сен  2 10:12 /etc/ssh/ssh_host_rsa_key.pub


Далее необходимо рассказать демону SSH о полученном этом сертификате хоста. Здесь есть три шага:
   - Изменить файл конфигурации SSH-демона, указать новый RSA-сертификат хоста.
   - Добавьть общесистемный файл ``known_hosts`` с открытым ключом Центра Сертификации ``ca_host_key.pub``, чтобы  SSH-демон мог проверять сертификаты.
   - Перезапустить/перезагрузить SSH.
    
В файл конфигурации SSH-демона, ``/etc/ssh/sshd_config``, необходимо добавить следующую строку:

        ### Host certificate
        HostCertificate /etc/ssh/ssh_host_rsa_key-cert.pub

, а затем перезагрузить SSH-демон командой:
        
        systemctl restart ssh

Далее необходимо создать общесистемный файл ``/etc/ssh/ssh_known_hosts``. В этом файле необходимо разместить содержимое открытого ключа ``CA``, файла ``ca_host_key.pub`` и некоторые другие данные, указывающие на то, что для проверки подписи Центра Cертификации для указанных доменов можно использовать указанный открытый ключ. Вот как это должно выглядеть:

        root@ca:/etc/ssh_ca# cat /etc/ssh/ssh_known_hosts 
        @cert-authority *.local ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQDLFAbnKtSwYjBLqjO0Djv6leiFL0w4G2rGBtY8w9+73NJuIic9J4hilSIpjEJeRnAIsdnNKuarfQE5o98goPTfh6J3ZmhSaeayJTJ/KYJMS8FJhtQPEtFhxNkoGnAKRoLBRRcjL5Rne83HHSVw0eh7hJHytCZCqr6dMfSH1BF/ZoAJ8ckZuZT1mUSWGoQPynx9OwozZTck/DHN9hNnj+PnCS4E3k5ECQOCiHxJyUgbotuVmw1HziHqAoOAvg+P8p9uM455LXW4jjx88fP6rjdOAinkzTlizVFK53ObPOUw1NnLyjq84UAzx6oPwH3KAcH3yfyBnzCfQGmsBbdQEpwAHtdNTO2EhqZMB/Q0B0U++ldMJ9Kufm3P4cLGhAv/pgfJAt8011w0GheEAngtpCNz/KomFss6g0wWWoyPzaEacYiIfMM1Z0qu3bjO/RXpSxlIN4J8UGc8xcjS8ZYHeH5nS5euqcGFzU+1AjzeFevvgvCFJcVEMJd10yRPt3secfmUWpD1WQzD/gCfAWanxx4A8L8ML5WD6mrLVPum5Ojtoaa5PBU9gFXw3XUPRRZeITx384PM3Iwr+dLZH/ZKJ+7wmG+Ljpo1VND+1M9aAJQ7DG/tpdv3Dhv/A3JJA5eUiwe/6gNjsYSefT4jGs8TYeWLyp6cjE3uZw3SGr6U8/c/fQ==


### Генерация и распределение сертификатов хостов 

Для генерации сертификатов хостов, можно повторить те же процессы (выше), а именно:

 1. Собрать открытые ключи хостов с серверов ``server1`` и ``server2``,  работающих под управлением SSHD.
 2. Подписать с помощью ключа ``CA`` открытые RSA-ключи хостов серверов и перенести полученные сертификаты на сервера.
 3. Создать файл ``/etc/ssh/ssh_known_hosts`` на каждом сервере.
 4. Скопировать содержимое открытого ключа хоста Центра Сертификации ``ca_host_key.pub`` в файл ``/etc/ssh/ssh_known_hosts`` на каждом сервере.
 5. Скопировать открытый ключ подписи сертификатов пользователей ``ca_user_key.pub`` на каждый сервер.
 6. Изменить файлы конфигурации SSHD ``/etc/ssh/sshd_config`` на каждом сервере.
 7. Перезапустить SSH-демоны на каждом сервере.
 
Напрмер, для сервера ``server1`` (для ``server2`` аналогично).

Создадим временную директорию ``keys`` на сервере ``CA`` для временного хранения сертификтов хостов и пользователей. Выполним это командой: 
    
        mkdir keys


Далее перенесем открытый ключ хоста ``ssh_host_rsa_key.pub`` в директорию ``keys`` из сервера ``server1`` командой:

        scp u1@u1.local:/etc/ssh/ssh_host_rsa_key.pub /home/ca/keys/ssh_host_rsa_key.pub

Чтобы создать сертификат хоста подписанный ``CA``, необходимо выполнить команду:

        ssh-keygen -s ca_host_key -I u1_host_key.local -h -n u1,u1.local -V -2w:+54W /home/ca/keys/ssh_host_rsa_key.pub


После получения сертификата хоста, необходимо перенести сертификат обратно на сервер ``server1`` командой:

        scp /home/ca/keys/ssh_host_rsa_key-cert.pub root@u1.local:/etc/ssh/
        
Вместе с сертификатом хоста, необходимо скопировать открытый ключ подписи сертификатов пользователей ``ca_user_key.pub`` на сервер:

        scp /etc/ssh_ca/ca_user_key.pub root@u1.local:/etc/ssh/

Далее на сервере ``server1`` необходимо установить в конфиге SSH-демона путь к полученному от ``CA`` сертификату и путь к ключу проверки подписей пользователей, то есть в файл ``/etc/ssh/sshd_config``, необходимо добавить следующуие строки:
    
        ### Host certificate
        HostCertificate /etc/ssh/ssh_host_rsa_key-cert.pub


        ### User CA certificate 
        TrustedUserCAKeys /etc/ssh/ca_user_key.pub

, а затем перезагрузить SSH-демон командой:
        
        sudo systemctl restart ssh
        

Далее необходимо создать общесистемный файл ``/etc/ssh/ssh_known_hosts``. В этом файле необходимо разместить содержимое открытого ключа ``CA``, файла ``ca_host_key.pub`` и некоторые другие данные, указывающие на то, что для проверки подписи Центра Cертификации для указанных доменов можно использовать указанный открытый ключ. Вот как это должно выглядеть:

        root@ca:/etc/ssh_ca# cat /etc/ssh/ssh_known_hosts 
        @cert-authority *.local ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQDLFAbnKtSwYjBLqjO0Djv6leiFL0w4G2rGBtY8w9+73NJuIic9J4hilSIpjEJeRnAIsdnNKuarfQE5o98goPTfh6J3ZmhSaeayJTJ/KYJMS8FJhtQPEtFhxNkoGnAKRoLBRRcjL5Rne83HHSVw0eh7hJHytCZCqr6dMfSH1BF/ZoAJ8ckZuZT1mUSWGoQPynx9OwozZTck/DHN9hNnj+PnCS4E3k5ECQOCiHxJyUgbotuVmw1HziHqAoOAvg+P8p9uM455LXW4jjx88fP6rjdOAinkzTlizVFK53ObPOUw1NnLyjq84UAzx6oPwH3KAcH3yfyBnzCfQGmsBbdQEpwAHtdNTO2EhqZMB/Q0B0U++ldMJ9Kufm3P4cLGhAv/pgfJAt8011w0GheEAngtpCNz/KomFss6g0wWWoyPzaEacYiIfMM1Z0qu3bjO/RXpSxlIN4J8UGc8xcjS8ZYHeH5nS5euqcGFzU+1AjzeFevvgvCFJcVEMJd10yRPt3secfmUWpD1WQzD/gCfAWanxx4A8L8ML5WD6mrLVPum5Ojtoaa5PBU9gFXw3XUPRRZeITx384PM3Iwr+dLZH/ZKJ+7wmG+Ljpo1VND+1M9aAJQ7DG/tpdv3Dhv/A3JJA5eUiwe/6gNjsYSefT4jGs8TYeWLyp6cjE3uZw3SGr6U8/c/fQ==
        


### Генерация и распределение сертификатов пользователей

Аналогично тому, как сертификаты хостов однозначно идентифицируют хосты для клиентов SSH, сертификаты пользователей используются для идентификации пользователей SSH для удаленных демонов.

Для генерации пользовательского сертификата, на виртуальной машине ``server2`` необходимо сгенерировать пользовательские RSA-ключи:

        ssh-keygen -t rsa -b 2048 -f ~/.ssh/id_rsa 

далее скопировать открытый ключ пользователя ``u2`` на сервер ``CA``: 

        scp /home/u2/.ssh/id_rsa.pub ca@ca.local:/home/ca/keys/

, на сервере ``CA`` выполнить команду: 

        ssh-keygen -s ca_user_key -I u2 -n u2.local,u1.local,ca.local -V -1w:+52w /home/ca/keys/id_rsa.pub
        
полученый сертификат скопировать обратно на сервер ``server2``:

        scp /home/ca/keys/id_rsa-cert.pub u2@u2.local:/home/u2/.ssh/

Необходимо убедиться, что на хосте, в конфиге SSH-демона ``/etc/ssh/sshd_config`` установлено поле:
       
       #
       TrustedUserCAKeys /etc/ssh/ca_user_key.pub
       #
       #

, если не установлено, убедться, что файл ``/etc/ssh/ca_user_key.pub``существует и установть это поле, презгрузить SSH-демон.


***Это необходимо для проверки пользовательских сертификатов, которые были предявлены пользователями при авторизации***


Далее необходимо создать общесистемный файл /etc/ssh/ssh_known_hosts. В этом файле необходимо разместить содержимое открытого ключа CA, файла ca_host_key.pub и некоторые другие данные, указывающие на то, что для проверки подписи Центра Cертификации для указанных доменов можно использовать указанный открытый ключ. Вот как это должно выглядеть:

    root@ca:/etc/ssh_ca# cat /etc/ssh/ssh_known_hosts 
    @cert-authority *.local ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQDLFAbnKtSwYjBLqjO0Djv6leiFL0w4G2rGBtY8w9+73NJuIic9J4hilSIpjEJeRnAIsdnNKuarfQE5o98goPTfh6J3ZmhSaeayJTJ/KYJMS8FJhtQPEtFhxNkoGnAKRoLBRRcjL5Rne83HHSVw0eh7hJHytCZCqr6dMfSH1BF/ZoAJ8ckZuZT1mUSWGoQPynx9OwozZTck/DHN9hNnj+PnCS4E3k5ECQOCiHxJyUgbotuVmw1HziHqAoOAvg+P8p9uM455LXW4jjx88fP6rjdOAinkzTlizVFK53ObPOUw1NnLyjq84UAzx6oPwH3KAcH3yfyBnzCfQGmsBbdQEpwAHtdNTO2EhqZMB/Q0B0U++ldMJ9Kufm3P4cLGhAv/pgfJAt8011w0GheEAngtpCNz/KomFss6g0wWWoyPzaEacYiIfMM1Z0qu3bjO/RXpSxlIN4J8UGc8xcjS8ZYHeH5nS5euqcGFzU+1AjzeFevvgvCFJcVEMJd10yRPt3secfmUWpD1WQzD/gCfAWanxx4A8L8ML5WD6mrLVPum5Ojtoaa5PBU9gFXw3XUPRRZeITx384PM3Iwr+dLZH/ZKJ+7wmG+Ljpo1VND+1M9aAJQ7DG/tpdv3Dhv/A3JJA5eUiwe/6gNjsYSefT4jGs8TYeWLyp6cjE3uZw3SGr6U8/c/fQ==


### Тест

        u2@u2:~$ ssh-keygen -L -f  .ssh/id_rsa-cert.pub
        .ssh/id_rsa-cert.pub:
                Type: ssh-rsa-cert-v01@openssh.com user certificate
                Public key: RSA-CERT SHA256:jFzZ+vvrqJ8FwGrAbl3X19u0nU8K2hyLkCYjsRf93bY
                Signing CA: RSA SHA256:dKA0isiVryPe1TQngzi5OCd/ZHbFAlUlvwlC2e2StkY
                Key ID: "u2"
                Serial: 0
                Valid: from 2019-08-26T15:25:59 to 2020-08-31T15:25:59
                Principals: 
                        u2
                        u1
                        ca
                Critical Options: (none)
                Extensions: 
                        permit-X11-forwarding
                        permit-agent-forwarding
                        permit-port-forwarding
                        permit-pty
                        permit-user-rc


        u2@u2:~$ ssh -i .ssh/id_rsa-cert.pub u1@u1.local
        Welcome to Ubuntu 18.04.3 LTS (GNU/Linux 5.0.0-23-generic x86_64)

         * Documentation:  https://help.ubuntu.com
         * Management:     https://landscape.canonical.com
         * Support:        https://ubuntu.com/advantage


         * Canonical Livepatch is available for installation.
           - Reduce system reboots and improve kernel security. Activate at:
             https://ubuntu.com/livepatch

        84 packages can be updated.
        53 updates are security updates.

        Your Hardware Enablement Stack (HWE) is supported until April 2023.
        Last login: Mon Sep  2 20:26:20 2019 from 192.168.56.112


### Ресурсы: 

1. https://access.redhat.com/documentation/en-us/red_hat_enterprise_linux/6/html/deployment_guide/sec-support_for_openssh_certificates
2. https://umbrielsecurity.wordpress.com/2017/03/07/certificate-based-ssh-auth/